<a href="https://colab.research.google.com/github/Arseny20/robustLLM/blob/dataset/get_logs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U datasets
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import torch
import json
import re
from datasets import load_dataset


classifier = pipeline("zero-shot-classification",
                     model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli",
                     device=0 if torch.cuda.is_available() else -1)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

Device set to use cuda:0


In [2]:
# Login using e.g. `huggingface-cli login` to access this dataset
df = load_dataset("ai4privacy/pii-masking-400k")["train"]
df

README.md: 0.00B [00:00, ?B/s]

1en.jsonl:   0%|          | 0.00/84.8M [00:00<?, ?B/s]

de.jsonl:   0%|          | 0.00/82.7M [00:00<?, ?B/s]

es.jsonl:   0%|          | 0.00/42.5M [00:00<?, ?B/s]

fr.jsonl:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

...

it.jsonl:   0%|          | 0.00/19.8M [00:00<?, ?B/s]

nl.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/325517 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/81379 [00:00<?, ? examples/s]

Dataset({
    features: ['source_text', 'locale', 'language', 'split', 'privacy_mask', 'uid', 'masked_text', 'mbert_tokens', 'mbert_token_classes'],
    num_rows: 325517
})

In [13]:
def edit_json(input: str):
    json_dict = input
    json_new = list()
    for item in json_dict:
        label_mapping = {
            "PHONE": "Phone",
            "PASSWORD": "password",
            "USERNAME": "login",
            "EMAIL": "email",
            "GIVENNAME": "FIO",
            "SURNAME": "FIO"
        }
        original_label = item["label"]
        item["label"] = label_mapping.get(original_label, original_label)
        if item["label"] in ["Phone", "password", "login", "email", "FIO"]:
            json_new.append(item)
    return json_new


def edit_text(text: str):
    # Regular expression to match HTML tags
    clean_text = re.sub(r'<[^>]+>', '', text)

    # Optional: Replace common HTML entities
    html_entities = {
        '&nbsp;': ' ',
        '&amp;': '&',
        '&lt;': '<',
        '&gt;': '>',
        '&quot;': '"',
        '&apos;': "'"
    }

    for entity, replacement in html_entities.items():
        clean_text = clean_text.replace(entity, replacement)

    return clean_text.strip()
    return text

def word_to_ner(text, label_data):
    text = text.split()
    idx = 0
    new_list = list()
    mapping = ["Phone", "password", "login", "email", "FIO"]
    for word in text:
        if idx < len(label_data) and word == label_data[idx]["value"]:
            new_list.append(mapping.index(label_data[idx]["label"])+1)
            idx += 1
        else:
            new_list.append(0)
    return new_list


def tokens_marking(tokens, token_classes):
    token_mapping = {
        "B-PHONE": "B-Phone",
        "B-PASSWORD": "B-password",
        "B-USERNAME": "B-login",
        "B-EMAIL": "B-email",
        "B-GIVENNAME": "B-FIO",
        "B-SURNAME": "B-FIO",
        "I-PHONE": "I-Phone",
        "I-PASSWORD": "I-password",
        "I-USERNAME": "I-login",
        "I-EMAIL": "I-email",
        "I-GIVENNAME": "I-FIO",
        "I-SURNAME": "I-FIO"
    }

    # Convert map object to list to use it multiple times
    token_classes = list(map(lambda x: token_mapping.get(x, 'O'), token_classes))

    result = []
    for token, cls in zip(tokens, token_classes):
        result.append((token, cls))  # Fixed: Using tuple() requires a single iterable argument

    return result

In [18]:
labels = [
    "development_logs",
    "application_logs",
    "corporate_logs",
    "other_logs",
    "message",
    "not_logs",
]


texts = list()
masks = list()
indices = list()
tokens = list()

for i in range(3000): # здесь вставить кол-во записей
    result = classifier(df[i]["source_text"], labels, multi_label=False)
    if result['labels'][0] in labels[0:3]:
        print(f"{i:6}/{df.shape[0]} {result['labels'][0]:15} {df[i]['source_text'][:100]}...")
        json_to_insert = edit_json(df[i]['privacy_mask'])
        if json_to_insert:
            print(json_to_insert)
            text_to_insert = edit_text(df[i]['source_text'])
            ner_indices = word_to_ner(text_to_insert, json_to_insert)
            tokens_to_insert = tokens_marking(df[i]['mbert_tokens'], df[i]['mbert_token_classes'])
            texts.append(text_to_insert)
            masks.append(json_to_insert)
            indices.append(ner_indices)
            tokens.append(tokens_to_insert)

   188/325517 application_logs Application completeness check: <br> Name: Marah Peyrilles <br> Job Title: Therapist, drama <br> ZIP...
[{'label': 'FIO', 'start': 43, 'end': 48, 'value': 'Marah', 'label_index': 1}, {'label': 'FIO', 'start': 49, 'end': 58, 'value': 'Peyrilles', 'label_index': 1}]
   253/325517 corporate_logs  Examination notes residence: 880 Pegglesworth, GL54. Records show 678 606 7954 and financial details...
   254/325517 corporate_logs  Doc details: 800 The Vintage Pair, ZIP GL54 2RJ. Age listed: 36. Social documentation 630.722.6794. ...
   315/325517 corporate_logs  Therapy check-in: Client Alfried Wohlgenannt, residing at White Springs, FL. Concerns over height (1...
[{'label': 'FIO', 'start': 25, 'end': 32, 'value': 'Alfried', 'label_index': 1}, {'label': 'FIO', 'start': 33, 'end': 44, 'value': 'Wohlgenannt', 'label_index': 1}]
   477/325517 development_logs The developmental evaluation of Niranjala Conejera focuses on their ability to engage socially and p...
[{

In [19]:
new_data = pd.DataFrame({
    "source_text": texts,
    "privacy_mask": masks,
    "ner_indices": indices,
    "tokens": tokens
})
new_data.to_csv('logs_data.csv', index=None)
new_data.head(10)

,source_text,privacy_mask,ner_indices,tokens
0,Application completeness check: Name: Marah P...,"[{'label': 'FIO', 'start': 43, 'end': 48, 'val...","[0, 0, 0, 0, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(Application, O), (complete, O), (##ness, O),..."
1,"Therapy check-in: Client Alfried Wohlgenannt, ...","[{'label': 'FIO', 'start': 25, 'end': 32, 'val...","[0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(Therapy, O), (check, O), (-, O), (in, O), (:..."
2,The developmental evaluation of Niranjala Cone...,"[{'label': 'FIO', 'start': 32, 'end': 41, 'val...","[0, 0, 0, 0, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(The, O), (development, O), (##al, O), (evalu..."
3,Mrs Vlada's developmental delay noted. Residen...,"[{'label': 'FIO', 'start': 4, 'end': 9, 'value...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(Mrs, O), (Vlad, B-FIO), (##a, I-FIO), (', O)..."
4,fatima.yavru09's medication plan overseen by B...,"[{'label': 'login', 'start': 3, 'end': 17, 'va...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(<, O), (p, O), (>, O), (fat, B-login), (##im..."
5,2 Support Plan Document outlining indivi...,"[{'label': 'FIO', 'start': 67, 'end': 72, 'val...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(2, O), (Support, O), (Plan, O), (Document, O..."
6,2. Export Compliance Report\nAssessment of the...,"[{'label': 'login', 'start': 112, 'end': 118, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(2, O), (., O), (Export, O), (Com, O), (##pli..."
7,Fund Performance: User jhdzvmhiyonunahx273 wit...,"[{'label': 'login', 'start': 38, 'end': 57, 'v...","[0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(<, O), (html, O), (>, O), (<, O), (body, O),..."
8,Analysis of fund 1629729153 in Princeton: Perf...,"[{'label': 'login', 'start': 113, 'end': 122, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(<, O), (html, O), (>, O), (<, O), (body, O),..."
9,2 Satellite Licensing Application Complete lic...,"[{'label': 'password', 'start': 238, 'end': 24...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(<, O), (html, O), (>, O), (<, O), (body, O),..."
